<h1 style="text-align: center;">Data Mining Project 2: Classification Analysis</h1>

<p style="text-align:center;">
    呂伯駿<br>
    Q56074085<br>
    NetDB<br>
    National Cheng Kung University<br>
    pclu@netdb.csie.ncku.edu.tw
</p>


## 1. Introduction




## 2. Environment

### 2.1 System Preferences

實驗環境如下：
- <b>Operating System</b>: macOS High Sierra (10.13.6)
- <b>CPU</b>: 1.3 GHz Intel Core i5
- <b>Memory</b>: 8 GB 1600 MHz DDR3
- <b>Programming Language</b> : Python 3.6.2

由於所用資料集大小皆不超過 4MB，實驗環境記憶體足以符合實驗需求。

### 2.2 Dataset

## 3. Implementation


## 4. Analysis

## 5. Conclusion



## 6. References